<a href="https://colab.research.google.com/github/kjgeorge88/colab-cfd-tests/blob/master/taylor_green_revisited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import time

# simulation parameters
scale  = 2               # set simulation size
NX     = 32*scale        # domain size
NY     = NX
NSTEPS = 200*scale*scale # number of simulation time steps
NMSG   = 50*scale*scale  # show messages every NMSG time steps
vis    = False           # show visualisation; set to False for performance measurements
NVIS   = NMSG            # show visualisation every NVIS time steps
tau    = 1               # relaxation time
u_max  = 0.04/scale      # maximum velocity
nu     = (2*tau-1)/6     # kinematic shear viscosity
rho0   = 1               # rest density
Re     = NX*u_max/nu     # Reynolds number; not used in the simulation itself

In [0]:
# Mesh parameters
x = np.arange(NX)+0.5
y = np.arange(NY)+0.5
[X, Y] = np.meshgrid(x,y)

In [0]:
# Taylor Green Vortex 
def taylorgreen(t, nu, rho0, u_max):
    kx = 2*np.pi/NX
    ky = 2*np.pi/NY
    td = 1/(nu*(kx*kx+ky*ky))
    u = -u_max*np.sqrt(ky/kx)*np.cos(kx*X)*np.sin(ky*Y)*np.exp(-t/td)
    v =  u_max*np.sqrt(kx/ky)*np.sin(kx*X)*np.cos(ky*Y)*np.exp(-t/td)
    P = -0.25*rho0*u_max*u_max*((ky/kx)*np.cos(2*kx*X)+(kx/ky)*np.cos(2*ky*Y))*np.exp(-2*t/td)
    rho = rho0+3*P
    return [rho, u, v, P]

In [0]:
# Initialise populations
[rho, u, v, P] = taylorgreen(0, nu, rho0, u_max)

print('Simulating Taylor-Green vortex decay')
print('      domain size:', str(NX) + 'x' + str(NY))
print('               nu:', nu)
print('              tau:', tau)
print('            u_max:', u_max)
print('             rho0:', rho0)
print('        timesteps:', NSTEPS)
print('       plot every:', NVIS)
print('    message every:', NMSG)
print('')

E = 0.5*(np.einsum('xy->', u*u) + np.einsum('xy->', v*v))
print(0, E, 0, 0, 0)

tstart = time.time()

In [0]:
import numba

@numba.stencil
def _diffusion(x,nu):
    return x[0,0] + 0.25*nu*(            x[0,-1] +
                             x[-1, 0] +            x[1, 0] +
                                         x[0, 1]             ) 

@numba.njit
def diffusion_cpu(x,nu):
    return _diffusion(x,nu)

In [0]:
x_cpu = np.ones((64, 64), dtype='int8')

%timeit diffusion_cpu(x_cpu,nu)

In [0]:
# Main loop
for t in range(1, NSTEPS+1):

    # Advance in time
    [rho, u, v, P] = taylorgreen(t, nu, rho0, u_max)

    #diffusion_cpu(u,nu)
    #diffusion_cpu(v,nu)
    #diffusion_cpu(P,nu)
    #rho = rho0 + 3*P 
     
    # Compare against the analytical solution every NMSG time steps
    if t % NMSG == 0:
        
        # Calculate analytical solution
        [rhoa, ua, va, Pa] = taylorgreen(t, nu, rho0, u_max)

        # Kinetic energy
        E = 0.5*(np.einsum('xy->', u*u) + np.einsum('xy->', v*v))

        # Sum square errors
        sumrhoe2 = np.einsum('xy->', (rho-rhoa)*(rho-rhoa))
        sumue2   = np.einsum('xy->', (u-ua)*(u-ua))
        sumve2   = np.einsum('xy->', (v-va)*(v-va))

        # Normalisation factors
        sumrhoa2 = np.einsum('xy->', (rhoa-rho0)*(rhoa-rho0))
        sumua2   = np.einsum('xy->', ua*ua)
        sumva2   = np.einsum('xy->', va*va)

        # L2 norms
        L2rho = np.sqrt( sumrhoe2  / sumrhoa2 )
        L2ux  = np.sqrt( sumue2    / sumua2 )
        L2uy  = np.sqrt( sumve2    / sumva2 )

        print(t, E, L2rho, L2ux, L2uy)

    # Plot solution every NVIS time steps
    if vis and t % NVIS == 0:
          umag = np.sqrt( np.einsum('axy->xy', u*u) ) / u_max
          plt.imshow(umag, extent=[0,1,0,1], vmin=0, vmax=1)
          bar = plt.colorbar()
          plt.streamplot(x/NX, y/NY, u[0,:,:], u[1,:,:], color=[1,1,1])
          plt.xlabel('$x/l_x$')
          plt.ylabel('$y/l_y$')
          bar.set_label('$|\mathbf{u}|/u_\mathrm{max}$')
          td = 1/(nu*(2*np.pi/NX)**2 + (2*np.pi/NY)**2)
          plt.title('flow field at $t/t_d = {0:.2f}$'.format(t/td))
          plt.show()

# Calculate performance information after the simulation is finished
runtime = time.time() - tstart
nodes_updated = NSTEPS*NX*NY
speed = nodes_updated/(1e6*runtime)

print(' ----- performance information -----')
print('        timesteps:', NSTEPS);
print('          runtime:', runtime, '(s)')
print('            speed:', speed, '(Mlups)')